In [ ]:
import numpy as np

class ThompsonSamplingLinear:
    
    def __init__(self, n_actions, n_features, alpha=1.0):
        self.n_actions = n_actions
        self.n_features = n_features
        self.alpha = alpha
        
        # Prior mean (initially zero)
        self.mu = np.zeros(n_features)
        
        # Prior covariance (identity matrix scaled by alpha)
        self.Sigma = np.eye(n_features) * alpha
    
    def select_action(self, action_features):
        # Sample from the posterior distribution
        theta_sampled = np.random.multivariate_normal(self.mu, self.Sigma)
        
        # Compute expected reward for each action based on the sampled theta
        expected_rewards = np.maximum(0, action_features.dot(theta_sampled))
        
        # Select the action with the highest expected reward
        return np.argmax(expected_rewards)
    
    def update(self, chosen_action_features, reward):
        # Update posterior distribution using Bayesian linear regression
        x = chosen_action_features.reshape(-1, 1)
        
        # Update the covariance matrix
        self.Sigma = np.linalg.inv(np.linalg.inv(self.Sigma) + x.dot(x.T))
        
        # Update the mean vector
        self.mu = self.Sigma.dot(np.linalg.inv(self.Sigma).dot(self.mu) + reward * x.flatten())

# Example usage
if __name__ == "__main__":
    np.random.seed(42)  # For reproducibility

    n_actions = 10
    n_features = 2048
    ts_linear = ThompsonSamplingLinear(n_actions, n_features)
    
    # Assume we have some action features and true rewards
    action_features = np.random.randn(n_actions, n_features)  # Random features for each action
    true_theta = np.random.randn(n_features)  # True underlying linear relationship
    rewards = action_features.dot(true_theta) + np.random.randn(n_actions) * 0.1  # Rewards with noise

    for _ in range(100):  # Simulate 100 iterations
        chosen_action = ts_linear.select_action(action_features)
        ts_linear.update(action_features[chosen_action], rewards[chosen_action])
        print(f"Selected action {chosen_action}, Reward: {rewards[chosen_action]:.4f}")


In [2]:
import numpy as np
from scipy.linalg import cholesky, solve_triangular

n = 3048  # Dimension of the matrix

# Generate a random positive-definite matrix
A = np.random.randn(n, n)
A = A.T @ A + np.eye(n)  # Ensure positive-definiteness

# Cholesky decomposition
%timeit L = cholesky(A, lower=True)

# Matrix inversion
%timeit A_inv = np.linalg.inv(A)


225 ms ± 9.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.22 s ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
